In [28]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from kmodes.kmodes import KModes
from scipy.stats import chisquare
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score , classification_report
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("poisonous_mushroom_dataset.csv")

In [3]:
df.describe

<bound method NDFrame.describe of      class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0        p         x           s         n       t    p               f   
1        e         x           s         y       t    a               f   
2        e         b           s         w       t    l               f   
3        p         x           y         w       t    p               f   
4        e         x           s         g       f    n               f   
...    ...       ...         ...       ...     ...  ...             ...   
8119     e         k           s         n       f    n               a   
8120     e         x           s         n       f    n               a   
8121     e         f           s         n       f    n               a   
8122     p         k           y         n       f    y               f   
8123     e         x           s         n       f    n               a   

     gill-spacing gill-size gill-color  ... stalk-surface-below-r

In [4]:
# Set random seed for reproducibility
np.random.seed(42)
random_rows = df.sample(n=10)
random_rows

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
1971,e,f,f,n,f,n,f,w,b,h,...,f,w,w,p,w,o,e,n,s,g
6654,p,f,s,e,f,y,f,c,n,b,...,s,p,p,p,w,o,e,w,v,l
5606,p,x,y,n,f,f,f,c,n,b,...,s,w,p,p,w,o,e,w,v,l
3332,e,f,y,g,t,n,f,c,b,n,...,s,g,p,p,w,o,p,n,y,d
6988,p,f,s,e,f,s,f,c,n,b,...,s,p,p,p,w,o,e,w,v,l
5761,p,x,y,n,f,y,f,c,n,b,...,s,w,p,p,w,o,e,w,v,l
5798,p,x,s,g,t,f,f,c,b,h,...,f,w,w,p,w,o,p,h,s,u
3064,p,x,y,y,f,f,f,c,b,g,...,k,n,b,p,w,o,l,h,y,p
1811,e,f,f,n,t,n,f,c,b,n,...,s,g,w,p,w,o,p,k,v,d
3422,e,f,y,n,t,n,f,c,b,n,...,s,w,w,p,w,o,p,k,y,d


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

No missing values for now 

In [6]:
df.duplicated().sum()

0

No duplicates 

In [7]:
for col in df.columns:
    if df[col].dtype == "object":
        print(f"{col}: {df[col].value_counts()}")

class: class
e    4208
p    3916
Name: count, dtype: int64
cap-shape: cap-shape
x    3656
f    3152
k     828
b     452
s      32
c       4
Name: count, dtype: int64
cap-surface: cap-surface
y    3244
s    2556
f    2320
g       4
Name: count, dtype: int64
cap-color: cap-color
n    2284
g    1840
e    1500
y    1072
w    1040
b     168
p     144
c      44
u      16
r      16
Name: count, dtype: int64
bruises: bruises
f    4748
t    3376
Name: count, dtype: int64
odor: odor
n    3528
f    2160
y     576
s     576
a     400
l     400
p     256
c     192
m      36
Name: count, dtype: int64
gill-attachment: gill-attachment
f    7914
a     210
Name: count, dtype: int64
gill-spacing: gill-spacing
c    6812
w    1312
Name: count, dtype: int64
gill-size: gill-size
b    5612
n    2512
Name: count, dtype: int64
gill-color: gill-color
b    1728
p    1492
w    1202
n    1048
g     752
h     732
u     492
k     408
e      96
y      86
o      64
r      24
Name: count, dtype: int64
stalk-shape: stalk

## bruises gill-attachment gill-spacing gill-size stalk-shape are binary <br> 
all  values in veil-type are identical 

Checking whtere ? is in values or not 

In [8]:
for i in df.columns:
    print(i,len(df[df[i] == '?']))

class 0
cap-shape 0
cap-surface 0
cap-color 0
bruises 0
odor 0
gill-attachment 0
gill-spacing 0
gill-size 0
gill-color 0
stalk-shape 0
stalk-root 2480
stalk-surface-above-ring 0
stalk-surface-below-ring 0
stalk-color-above-ring 0
stalk-color-below-ring 0
veil-type 0
veil-color 0
ring-number 0
ring-type 0
spore-print-color 0
population 0
habitat 0


stalk-root has 2480 missing values. around %25 are missing in this column. 

since we have 22 features, I will drop this one for now

## Dropping the useless columns

In [9]:
df = df.drop(["stalk-root" , "veil-type"], axis= 1)

## Label encoding of the columns

In [10]:
label_encoder = LabelEncoder()
# List of columns to encode
labelEncode = ["class", 'bruises', 'gill-attachment', 'gill-spacing', 'gill-size', 'stalk-shape']

# Applying LabelEncoder to each column in the list
for column in labelEncode:
    df[column] = label_encoder.fit_transform(df[column])

## One-hot encode the others: 

In [11]:
OneHotEncode = [col for col in df.columns if col not in labelEncode]
# Apply one-hot encoding
df = pd.get_dummies(df, columns=OneHotEncode)

In [12]:
df.loc[: , df.columns] = df.loc[: , df.columns].astype("int")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10148\422728190.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 1 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[: , df.columns] = df.loc[: , df.columns].astype("int")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10148\422728190.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[: , df.columns] = df.loc[: , df.columns].astype("int")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10148\422728190.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 1 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  

## outlier detection using K-modes 

In [13]:
# Apply k-modes clustering
km = KModes(n_clusters=3, init='Huang', n_init=5, verbose=1)
clusters = km.fit_predict(df)

# Find clusters with few points
cluster, counts = np.unique(clusters, return_counts=True)
outlier_clusters = cluster[counts < 801]  # Example threshold
outlier_data = df[np.isin(clusters, outlier_clusters)]

print("Outlier data based on clustering:")
print(outlier_data)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 3295, cost: 92202.0
Run 1, iteration: 2/100, moves: 1309, cost: 90426.0
Run 1, iteration: 3/100, moves: 0, cost: 90426.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 2425, cost: 91826.0
Run 2, iteration: 2/100, moves: 613, cost: 91826.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 1758, cost: 85370.0
Run 3, iteration: 2/100, moves: 571, cost: 84890.0
Run 3, iteration: 3/100, moves: 18, cost: 84878.0
Run 3, iteration: 4/100, moves: 0, cost: 84878.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 2990, cost: 90512.0
Run 4, iteration: 2/100, moves: 299, cost: 90236.0
Run 4, iteration: 3/100, moves: 269, cost: 90236.0
Init: initializing centroids
Init: initializing clusters
Starting it

### K-modes detected no outliers even with 800 member cluster

## Feature selection

I will use Two approaches for findig features: <br>
- PCA
- Mutual Information

I will evaluate each method via RF training and finding R2

I will evaluate each method via RF training and finding R2

## PCA

In [14]:
# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(df.drop(["class"], axis=1), df["class"], train_size=0.6, test_size=0.4,random_state=0)

In [15]:
sm = SMOTE(random_state=2)

print("\nClass 1 before Over Sampling --> ", sum(y_train == 1))
print("\nClass 0 before Over Sampling --> ", sum(y_train == 0))

X_train, y_train = sm.fit_resample(X_train, y_train)

print("\nThe shape of X after Over Sampling -->", X_train.shape)
print("\nThe shape of Y after Over Sampling -->", y_train.shape)

print("\nClass 1 after Over Sampling --> ", sum(y_train == 1))
print("\nClass 0 after Over Sampling --> ", sum(y_train == 0))
print("\n")


Class 1 before Over Sampling -->  2365

Class 0 before Over Sampling -->  2509

The shape of X after Over Sampling --> (5018, 106)

The shape of Y after Over Sampling --> (5018,)

Class 1 after Over Sampling -->  2509

Class 0 after Over Sampling -->  2509




In [16]:
# Define the pipeline with PCA and RandomForest
pipeline = Pipeline([
    ('scaler', StandardScaler()),     
    ('pca', PCA(n_components=15)),     
    ('rf', RandomForestClassifier(n_estimators=100, random_state=0))  
])

In [17]:
# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=15)),
                ('rf', RandomForestClassifier(random_state=0))])

In [18]:
# Preprocessing of validation data, get predictions
preds = pipeline.predict(X_valid)

In [19]:
accuracy = accuracy_score(y_valid, preds)
print("accuracy Score:", accuracy)

accuracy Score: 0.9993846153846154


In [20]:
report = classification_report(y_valid, preds)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1699
           1       1.00      1.00      1.00      1551

    accuracy                           1.00      3250
   macro avg       1.00      1.00      1.00      3250
weighted avg       1.00      1.00      1.00      3250



It seems only 15 PC is enough for prediction 


In [21]:
# Initialize PCA and apply
scaler = StandardScaler()
scaled_df= scaler.fit_transform(df.drop('class', axis=1))

In [22]:
pca = PCA(n_components=15)
principal_components = pca.fit_transform(scaled_df)

# Create a DataFrame with the principal components
pc_df = pd.DataFrame(data=principal_components,
                     columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC113', 'PC14', 'PC15'])


In [23]:
pc_df

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC113,PC14,PC15
0,-3.165783,-1.051409,-0.672080,-0.032635,-1.276557,-1.552267,2.500430,-0.709673,4.526044,2.010648,-2.700675,-2.016022,-2.487903,-0.735656,0.632026
1,-2.804768,-2.160630,0.715157,0.651840,-1.751403,-0.884633,3.601780,-0.273448,-0.145431,-2.291461,-0.727738,0.927716,0.678870,-1.364452,-0.735875
2,-4.038488,-1.898541,0.735648,0.696377,-1.698853,-0.435529,5.756937,-0.584976,-1.810376,-2.279888,0.721769,0.418146,1.828619,-0.799691,-2.402833
3,-3.104999,-1.719959,-0.418626,0.005670,-0.507315,-1.199590,3.003978,-0.527781,4.239557,2.049851,-2.245828,-1.212912,-2.141227,-0.574853,0.382753
4,-2.383044,-1.174443,-0.898482,0.550865,-2.815379,-2.291460,-2.862596,0.062333,-0.798720,-1.390564,-1.102217,0.640871,0.607446,0.146025,-0.219045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,-4.847178,13.180495,11.218230,-3.765815,-2.334894,-0.392869,-0.650115,-0.263463,0.272741,-0.285688,-0.138322,0.367327,0.158728,-0.419981,-0.273314
8120,-4.421089,12.007107,10.711193,-5.092680,-2.056571,-1.621009,-0.116227,-0.562336,-0.367861,0.609588,-0.240638,0.323385,2.107111,0.498458,0.318655
8121,-4.827645,11.002406,10.154401,-4.624062,-2.315564,0.023639,-0.893363,-0.710757,0.972516,-0.846507,0.245948,0.071032,-0.441592,1.338744,-0.523799
8122,1.372839,3.767662,-3.712245,-0.016275,-0.894817,-0.521866,1.023423,0.105349,0.019684,-0.308948,0.451396,0.470232,-1.384907,0.201915,0.694463


In [24]:
concat_df = pd.concat([pc_df, df["class"]], axis=1)

## Now we are good to go for LR

In [26]:
# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(concat_df.drop(["class"], axis=1), concat_df["class"], train_size=0.7, test_size=0.3,random_state=0)

In [27]:
sm = SMOTE(random_state=2)

print("\nClass 1 before Over Sampling --> ", sum(y_train == 1))
print("\nClass 0 before Over Sampling --> ", sum(y_train == 0))

X_train, y_train = sm.fit_resample(X_train, y_train)

print("\nThe shape of X after Over Sampling -->", X_train.shape)
print("\nThe shape of Y after Over Sampling -->", y_train.shape)

print("\nClass 1 after Over Sampling --> ", sum(y_train == 1))
print("\nClass 0 after Over Sampling --> ", sum(y_train == 0))
print("\n")


Class 1 before Over Sampling -->  2750

Class 0 before Over Sampling -->  2936

The shape of X after Over Sampling --> (5872, 15)

The shape of Y after Over Sampling --> (5872,)

Class 1 after Over Sampling -->  2936

Class 0 after Over Sampling -->  2936




In [31]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

In [32]:
# Initialize the Logistic Regression classifier
classifier = LogisticRegression()
# Fit the classifier to the training data
classifier.fit(X_train, y_train)
# Predict labels for the test data
preds = classifier.predict(X_valid)

In [33]:
report = classification_report(y_valid, preds)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99      1272
           1       0.99      0.99      0.99      1166

    accuracy                           0.99      2438
   macro avg       0.99      0.99      0.99      2438
weighted avg       0.99      0.99      0.99      2438



## Training LR with CV 

In [34]:
from sklearn.model_selection import cross_val_score

# Initialize Logistic Regression classifier model
classifier = LogisticRegression()

# Perform cross-validation
cv_scores = cross_val_score(classifier, X_train, y_train, cv=10)

# Calculate the mean and standard deviation of cross-validation scores
mean_cv_score = cv_scores.mean()
std_cv_score = cv_scores.std()

# Print the mean and standard deviation of cross-validation scores
print("Mean Cross-Validation Score:", mean_cv_score)
print("Standard Deviation of Cross-Validation Score:", std_cv_score)


Mean Cross-Validation Score: 0.990633510644462
Standard Deviation of Cross-Validation Score: 0.003163559810982977
